In [1]:
import threading
import time
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import pynq.lib.rgbled as rgbled 

led4 = rgbled.RGBLED(4)
btns = base.btns_gpio

In [ ]:
stop_event = threading.Event()

def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4)
    '''
    if n == 4:
        for i in range(int(t/2)):
            led4.on(2)
            time.sleep(d)
            led4.off()
            time.sleep(d)
    else:
        for i in range(t):
            base.leds[n].toggle()
            time.sleep(d)
        base.leds[n].off()
        
def button_monitor_t():
    while not stop_event.is_set():
        if btns.read() != 0:
            stop_event.set()

def philosopher_t(_l0, _l1, num):
    while not stop_event.is_set():
        has_l0 = False
        has_l1 = False
        
        if has_l1:
            _l1.release()
            has_l1 = False
        
        has_l0 = _l0.acquire(True)
        print("Philosopher {} has one chopstick".format(num))
        
        has_l1 = _l1.acquire(True)
        print("Philosopher {} is eating".format(num))
        
        blink(50, 0.02, num)
        
        if has_l0:
            _l0.release()
            has_l0 = False
        if has_l1:
            _l1.release()
            has_l1 = False
            
        print("Philosopher {} is napping".format(num))
        
        blink(15,0.1, num)


# Initialize and launch the threads
threads = []

locks = []
for i in range(5):
    locks.append(threading.Lock())

for i in range(4):
    t = threading.Thread(target=philosopher_t, args=(locks[i], locks[i+1], i))
    threads.append(t)
    t.start()

t = threading.Thread(target=philosopher_t, args=(locks[0], locks[4], 4))
threads.append(t)
t.start()
    
button_thread = threading.Thread(target=button_monitor_t)
button_thread.start()


for t in threads:
    name = t.name
    t.join()
    print('{} joined'.format(name))

button_thread.join()
print("button thread joined")
